In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from math import factorial
import pylab
pylab.rcParams['figure.figsize'] = (10, 6)

In [2]:
#
#
# This is a continuation of the data exploration started in summary_all.ipynb. The first
# notebook considered the data on the largest scale, aggregating all customers together.
# We now want to understand exactly what the customer data looks like to get a sense of how
# we can use it to make predictions.
#
#

In [5]:
#
# We throw away a small fraction of the training data for an exploratory playground
#
N = 10000 # full training set is 75000
train = pd.read_csv('order_products__train.csv')
oids = train['order_id'].unique()[:N]

#print train.groupby('order_id').get_group(1)
train = train[train['order_id'].isin(oids)]
#print train.groupby('order_id').get_group(1)

# get the corresponding users ids
orders = pd.read_csv('orders.csv')
uids = orders[orders['order_id'].isin(oids)]['user_id']
assert len(uids) == len(oids)

orders = orders[orders['user_id'].isin(uids)]

# now get the products contained in these orders
products = orders.merge(pd.read_csv('order_products__prior.csv'), on='order_id', how='left')
products = products.merge(pd.read_csv('products.csv'), on='product_id', how='left')
products = products.merge(pd.read_csv('departments.csv'), on='department_id', how='left')

print products.head()

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0    375303       30    prior             1          1                 13   
1    375303       30    prior             1          1                 13   
2    375303       30    prior             1          1                 13   
3   2156000       30    prior             2          1                 10   
4   3169631       30    prior             3          1                 22   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     NaN     21386.0                1.0        0.0   
1                     NaN     27839.0                2.0        0.0   
2                     NaN     45128.0                3.0        0.0   
3                     7.0     21386.0                1.0        1.0   
4                     7.0     21386.0                1.0        1.0   

                         product_name  aisle_id  department_id department  
0                          Smartwa

In [6]:
#
# Let's get crazy. Let our hair down a bit ...
#
x = products.groupby('department').apply(lambda x: sum(x['reordered']) / len(x))
print x.sort_values()

department
personal care      0.315587
pantry             0.348234
international      0.366351
other              0.389417
household          0.402363
missing            0.408463
canned goods       0.462662
dry goods pasta    0.465326
frozen             0.544682
breakfast          0.552753
babies             0.572886
snacks             0.572935
meat seafood       0.574865
alcohol            0.586011
bulk               0.610356
deli               0.614684
bakery             0.625252
pets               0.628368
beverages          0.652905
produce            0.653538
dairy eggs         0.672758
dtype: float64


In [ ]:
#
# Not surprising. The kind of product it is correlates to its "repurchasedness".
# We should include department_id as a feature in the next step. It's maybe not clear
# whether this contains *new* information. 
#